In [2]:
import numpy as np
import math
import pandas as pd
from collections import Counter
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [5]:
#Parâmetros do usuário (Definir entrada dos dados)
estat_ord = 'min' #Estatística para ordenamento
qtd_alg_nd = 1 #Quantidade máxima de algoritmos para separar o conjunto de possíves não-duplicadas
k = 3 #Tamanho da janela
orcamento = 100 #Ou uma porcentagem da base de dados
seed = 500

In [33]:
def geraF1(toClass, classificador): #toClass é o conjunto de treinamento
    
    f1 = 0
    
    if classificador == 'SVM':
        modelo = SVC(random_state = seed)
    elif classificador == 'DT':
        modelo = DecisionTreeClassifier(random_state = seed)
    
    #Separação do conjunto X e y
    XtoClass = toClass.iloc[:,1:]
#     print(XtoClass)
    ytoClass = toClass.duplicata
    
    #Divisão dos conjuntos de treino e teste (20% para esse último)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(XtoClass, ytoClass, test_size=0.20, random_state=seed)
    
    #Validação cruzada
    #This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.
    kfold = StratifiedKFold(n_splits=5, random_state=seed) #n_splits igual ao tamanho do conjunto de treino?
    cv_results = model_selection.cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='f1')
    f1 = cv_results.mean()
    std = cv_results.std()
    coef_var = std/f1
#     tamTreino = len(XtoClass)
    
    return f1, std, coef_var#, tamTreino

In [63]:
#geraTrainSet(weight_vector_dict_orig, dirDest, 'train' + '(' + algUtl + ')' + num + '.csv')


def geraTrainSet(ct, dir, file1):

    #ct --> vetor de similaridade + classe
    #cont = 0
    
    cols = list(ct.columns.values)
    cols.pop(cols.index('duplicata'))
    ct = ct[cols+['duplicata']]
    
#     trainSet  = open(dir+file1, 'ab')
#     trainSet  = open(dir+file1, 'wb')
    
    ct.to_csv(dir+file1, sep=';', index=False)        

#     writer_ts = csv.writer(trainSet, delimiter=';', quotechar = ' ')
 
#     for pesos in oracle_class_cache_set:

#         for k,v in dic.iteritems(): 
#             if pesos == v[1]:
#                 ids = k.split('~')

#                 if v[0]:
#                     #print 'Verdadeiro!'
#                     #lista = [cont] + ids + list(v[1]) + [1.0]
#                     lista = list(v[1]) + [1.0]
#                     writer_ts.writerow(lista)
#                 else:
#                     #print 'Falso!'
#                     #lista = [cont] + ids + list(v[1]) + [0.0]
#                     lista = list(v[1]) + [0.0]
#                     writer_ts.writerow(lista)
#                 #cont = cont + 1
#                 break
        
#     trainSet.close()

In [7]:
#Leitura do conjunto de pares conflitantes
pc = "../../csv/conjuntosDS/NAO_DA3.csv"
# pc = pd.read_csv(pc, sep=';') #pares conflitantes
pc = pd.read_csv(pc, sep=';', index_col=['elemento1', 'elemento2']) #pares conflitantes
# pc

In [100]:
pc_aa = pc.iloc[:, :5 ] #Conjunto onde serão aplicadas as janelas deslizantes
pc_vetores = pc.iloc[:, 5: ] #Conjunto base para compor o conjunto treinamento 

In [81]:
pc_aa.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
4975,5889,1.0,21,0.903782,1.000000,0.963424
5577,8990,1.0,20,0.942708,1.000000,0.974635
3475,8592,1.0,19,0.910085,0.980903,0.958294
3192,7923,1.0,20,0.950000,1.000000,0.979358
4395,8725,1.0,20,0.944444,0.984375,0.963168


In [82]:
pc_vetores.head()

,,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,
4975,5889,1.000000,1.0,1.000000,0.833333,1.000000,0.833333,1.000000
5577,8990,1.000000,1.0,1.000000,0.937500,1.000000,0.937500,1.000000
3475,8592,0.966667,1.0,0.888889,1.000000,0.857143,1.000000,0.857143
3192,7923,1.000000,1.0,0.900000,1.000000,1.000000,1.000000,1.000000
4395,8725,1.000000,1.0,0.888889,0.933333,1.000000,0.933333,1.000000


In [101]:

duplicata = pc_aa.loc[:, 'duplicata' ]
# cols = list(pc_aa.columns.values)
# cols.pop(cols.index('duplicata'))

# cols = list(pc_vetores.columns.values)
# pc_vetores = pc_vetores[['duplicata']+cols]

pc_aa.head()

pc_vetores = pd.concat([duplicata, pc_vetores], axis=1, ignore_index=False) #X aqui deve ser pc_vetores

# pc_vetores.insert(0,duplicata, None)
# pc_vetores

pc_vetores.head()
# duplicata

# pc_aa = pc.loc[:, :'med' ]
# pc_vetor = pc.loc[:, 'med': ] #Como referenciar a coluna vizinha à 'med'?

,,duplicata,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,,
4975,5889,1.0,1.000000,1.0,1.000000,0.833333,1.000000,0.833333,1.000000
5577,8990,1.0,1.000000,1.0,1.000000,0.937500,1.000000,0.937500,1.000000
3475,8592,1.0,0.966667,1.0,0.888889,1.000000,0.857143,1.000000,0.857143
3192,7923,1.0,1.000000,1.0,0.900000,1.000000,1.000000,1.000000,1.000000
4395,8725,1.0,1.000000,1.0,0.888889,0.933333,1.000000,0.933333,1.000000


In [9]:
#Separação do conjunto de pares conflitantes em dois a partir da quantidade de algoritmos 
#que aponta o par como possível duplicata 
#(conjunto A com quantidade de algoritmos = 1, com maioria composta por possíveis não-duplicatas,
#e conjunto B com quantidade de algoritmos > 1, contendo mais possíveis duplicatas que o conjunto A)
ndup = pc_aa.loc[pc['qtdAlg'] == qtd_alg_nd]
dup = pc_aa.loc[pc['qtdAlg'] != qtd_alg_nd]

#Ordenamento dos pares pela estatística selecionada
ndup = ndup.sort_values(estat_ord) 
dup = dup.sort_values(estat_ord, ascending=False)

In [10]:
ndup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
104289,105349,0.0,1,0.488872,0.488872,0.488872
102831,105349,0.0,1,0.492994,0.492994,0.492994
105349,105518,0.0,1,0.496958,0.496958,0.496958
103997,105349,0.0,1,0.504825,0.504825,0.504825
101480,107779,0.0,1,0.506742,0.506742,0.506742


In [11]:
dup.head()

,,duplicata,qtdAlg,min,max,med
elemento1,elemento2,,,,,
7008,8831,1.0,22,1.0,1.0,1.0
4481,6716,1.0,22,1.0,1.0,1.0
2004,3710,1.0,10,1.0,1.0,1.0
7201,8536,1.0,19,1.0,1.0,1.0
4384,5657,1.0,20,1.0,1.0,1.0


In [12]:
len(dup) + len(ndup)

1101

In [102]:
#Criação do dataframe que armazenará o conjunto de treinamento

conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)
# conj_treino.insert(0,'duplicata', None)
conj_treino

,duplicata,title,artist,track01,track02,track03,track10,track11


In [ ]:
#Percorrer o dataframe pelo tamanho da janela e comparar pares para seleção retornando o índice

#Acho que tenho q ?????

In [15]:
# jan_inic = 0 
# jan_fin = k

# #for orçamento ou f1 (Ajustar depois)

# #Indentar depois que ajustar o for
# grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
# id_ndup_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
# #     id_row_sel = grupo['min'].idxmin() #Retorna o índice da linha com menor valor da coluna ('min') especificada
    
# #Atualização da janela
# jan_inic = jan_inic + k
# jan_fin = jan_fin + k


In [18]:
# #A partir do índice retornado, selecionar par de ndup e dup,
# #remover deles e de pc_vetores (não necessário, só manter a posição atual em que está a janela)
# #e adicionar ao dataframe conj_treino

# #Isso tem que estar dentro de um loop 
# #(tanto na primeira situação em que se tem o povoamento inicial do conjunto treinamento
# #quanto nas iterações convencionais seguintes)

# duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

# vetor_sel = pc_vetores.loc[id_row_sel]
# conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
# conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino

# # type(x) #pandas.core.series.Series (Em que isso implica? Apenas um DataFrame sem cabeçalho?)
# # type(grupo) #pandas.core.frame.DataFrame
# conj_treino

In [ ]:
#Abordagem propriamente dita (Falta o povoamento inicial)

continua = True
f1_anterior = 0
f1_atual = 0
f1_svm = 0
f1_dt = 0

jan_inic = 0 
jan_fin = k

while continua:

    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
#         id_row_sel = tuple(map(tuple, id_row_sel))
#         print(type(id_row_sel))
#         id_row_sel = id_row_sel[0]
#         print(id_row_sel)
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    #     id_row_sel = grupo['min'].idxmin() #Retorna o índice da linha com menor valor da coluna ('min') especificada

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
#         id_row_sel = tuple(map(tuple, id_row_sel))
# #         print(dtype(id_row_sel))
#         id_row_sel = id_row_sel[0]

    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
    f1_svm = geraF1(conj_treino, 'SVM')
    f1_dt = geraF1(conj_treino, 'DT')
    
    f1_atual = (f1_svm + f1_dt)/2
    
    orcamento -= 1 #Têm que ser atualizado no povoamento inicial também
    
    if orcamento > 0 :
        
        if f1_atual > f1_anterior: #Selecionam-se novos pares para rotulação
            continua = True
        else:
            continua = False
            #Pensar a respeito: Máximo local, máximo global
            #Se a medida de qualidade for igual à anterior (por duas vezes?)
            #FALSE
    else:
        continua = False
    
    print("Quantidade de itens no conjunto de treinamento: {}".format(Counter(XtoClass)))
    
    dup = [i for i in conj_treino.duplicata if i == True]
    percentage = float(dup.count(True)/float(len(conj_treino))*100)
    print("{0}% de duplicatas.".format(str(percentage)[:6]))
                
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k

In [138]:
#Povoamento inicial

import sys

orcamento = 100
conj_treino = pd.DataFrame(columns=pc_vetores.columns.values)
# conj_treino.insert(0,'duplicata', None)

continua = True
f1_anterior = 0
f1_atual = 0
f1_svm = 0
f1_dt = 0

jan_inic = 0 
jan_fin = k

tamTreino = 0

cont = 0

entrouExcept = False

while cont < 10:

#     print("orçamento atual: {0}.".format(orcamento))

    print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    
    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    entrouExcept = False
    
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    
    except ValueError:
        
        entrouExcept = True
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        id_row_sel = tuple(map(tuple, id_row_sel))
        print(type(id_row_sel))
        id_row_sel = id_row_sel[0]
#         print(id_row_sel)
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par
    
#     print("tipoDoId: {3} - entrouExcept: {2} - id: {0} - duplicata: {1}.".format(id_row_sel, duplicata, entrouExcept, type(id_row_sel)))          

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
#     conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    entrouExcept = False
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    #     id_row_sel = grupo['min'].idxmin() #Retorna o índice da linha com menor valor da coluna ('min') especificada

    except ValueError:
        
        entrouExcept = True
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        id_row_sel = tuple(map(tuple, id_row_sel))
#         print(dtype(id_row_sel))
        id_row_sel = id_row_sel[0]

    except:
        print('Eita!')
        
    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par
          
#     print("tipoDoId: {3} - entrouExcept: {2} - id: {0} - duplicata: {1}.".format(id_row_sel, duplicata, entrouExcept, type(id_row_sel)))          

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
#     conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    orcamento -= 2 #Têm que ser atualizado no povoamento inicial também
    
    cont += 1
    
    
    
# print(conj_treino)

# ct = conj_treino

# cols = list(ct.columns.values)
# cols.pop(cols.index('duplicata'))
# ct = ct[cols+['duplicata']]

# print(ct)

dirDest = "./"

geraTrainSet(conj_treino, dirDest, 'train.csv')


indicesCT = conj_treino.index.values.tolist()

geral = pd.concat([pc_vetores,conj_treino])

teste = geral.drop(indicesCT, axis='rows')

# teste = geral.loc[np.setdiff1d(pc_vetores.index, conj_treino)]
# teste = geral.drop_duplicates(keep=False)

# groups = geral.groupby(level=geral.index.names)  
# teste = groups.last() # or .first()

# teste = geral.reset_index().drop_duplicates(subset='index', keep='first').set_index('index')

    
#Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')
    
f1_atual = (f1_svm + f1_dt)/2
std_atual = (std_svm + std_dt)/2
cv_atual = (cv_svm + cv_dt)/2



# print(f1_atual)
    

    
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

cv_atual = cv_atual*100
print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:6]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:6]))

# print("Número de duplicatas: {0} .".format(float(duplicatas.count(1.0))))
# print("Número de não-duplicatas: {0} .".format(float(naoduplicatas.count(0.0))))
                
    

orcamento: 100 - jan_inic: 0 - jan_fin: 3
orcamento: 98 - jan_inic: 3 - jan_fin: 6
orcamento: 96 - jan_inic: 6 - jan_fin: 9
orcamento: 94 - jan_inic: 9 - jan_fin: 12
orcamento: 92 - jan_inic: 12 - jan_fin: 15
orcamento: 90 - jan_inic: 15 - jan_fin: 18
orcamento: 88 - jan_inic: 18 - jan_fin: 21
orcamento: 86 - jan_inic: 21 - jan_fin: 24
orcamento: 84 - jan_inic: 24 - jan_fin: 27
orcamento: 82 - jan_inic: 27 - jan_fin: 30
Quantidade de itens no conjunto de treinamento: 20
F1: 0.8099999999999999
Desvio padrão: 0.22867962264113206
Coeficiente de variação: 32.75%.
45.0% de duplicatas.
55.000% de nao-duplicatas.


C:\Users\Diego\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [139]:
teste
print("Tamanho do conjunto pc_vetores: {0}.".format(len(pc_vetores)))
print("Tamanho do conjunto de treinamento: {0}.".format(len(conj_treino)))
print("Tamanho do conjunto de teste: {0}.".format(len(teste)))

Tamanho do conjunto pc_vetores: 1101.
Tamanho do conjunto de treinamento: 20.
Tamanho do conjunto de teste: 1081.


In [118]:
pc_vetores
conj_treino
geral
teste

,,duplicata,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,,
1818,7075,1.0,0.879167,1.000000,0.750000,0.750000,0.735294,0.750000,0.735294
1838,7934,1.0,1.000000,1.000000,1.000000,0.523810,1.000000,0.523810,1.000000
1858,9637,1.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1884,9542,1.0,1.000000,1.000000,0.777778,0.906250,0.769231,0.906250,0.769231
2004,3710,1.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2026,4601,1.0,0.600000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2139,3296,1.0,0.550000,1.000000,0.833333,0.842105,0.923077,0.842105,0.923077
2187,2418,1.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2188,8063,1.0,0.960000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [132]:
indicesCT = conj_treino.index.values
indicesPCVT = pc_vetores.index.values
# indicesCT
indicesPCVT

array([(4975, 5889), (5577, 8990), (3475, 8592), ..., (10652, 9640),
       (5912, 8949), (6568, 9557)], dtype=object)

In [129]:
cont = 0
for i in indicesCT:
    cont += 1
    print(pc_vetores.loc[i])

print(cont)

duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.500000
track02      0.454545
track03      0.818182
track10      0.454545
track11      0.818182
Name: (105349, 105518), dtype: float64
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (2004, 3710), dtype: float64
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.416667
track02      0.909091
track03      0.500000
track10      0.909091
track11      0.500000
Name: (105349, 3710), dtype: float64
duplicata    1.0
title        1.0
artist       0.0
track01      1.0
track02      1.0
track03      1.0
track10      1.0
track11      1.0
Name: (1858, 9637), dtype: float64
duplicata    0.000000
title        1.000000
artist       0.000000
track01      0.461538
track02      0.409091
track03      0.833333
track10      0.409091
track11      0.833333
Name: (101480, 104396), dtype: float64
duplicata    0.

In [51]:
print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))

orcamento: 80 - jan_inic: 30 - jan_fin: 33


In [52]:
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

continua = True
max_local = 0
igual = 0

print("Início")
print("orcamento: {0}".format(orcamento))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

while continua:
    
    f1_anterior = f1_atual

    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe

    #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    try:
    
        id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    
    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
        
    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #SELEÇÃO DA POSSÍVEL DUPLICATA
    #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    grupo = dup.iloc[jan_inic:jan_fin] # three rows of dataframe
    
    try:
    
        id_row_sel = grupo['qtdAlg'].idxmin() #Retorna o índice da linha com maior valor da coluna ('min') especificada

    except ValueError:
        
        #Seleciona um aleatoriamente
        a = grupo.sample(n = 1, random_state=seed) 
        id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada

    except:
        print('Eita!')

    #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
    duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par

    vetor_sel = pc_vetores.loc[id_row_sel]
    conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
    conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
    #Treina SVM e Decision Tree (justificar o pq) e verifica-se a média da medida de qualidade (f1, precision...)
    f1_svm, std_svm, cv_svm = geraF1(conj_treino, 'SVM')
    f1_dt, std_dt, cv_dt = geraF1(conj_treino, 'DT')

    f1_atual = (f1_svm + f1_dt)/2
    std_atual = (std_svm + std_dt)/2
    cv_atual = (cv_svm + cv_dt)/2
    
    cv_atual = cv_atual*100
    
    orcamento -= 2 #Têm que ser atualizado no povoamento inicial também
    
#     print("F1 anterior: {0} - F1 atual: {0}".format(f1_anterior, f1_atual))
    
#     if orcamento > 0 :
        
#         if f1_atual >= f1_anterior: #Selecionam-se novos pares para rotulação
#             print('Entrou no primeiro if!')
#             print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
#             continua = True
# #             break
#         else:
#             print("Aqui: F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
# #             print('Entrou no else do primeiro if!')
            
#             continua = False
#             break
#             #Pensar a respeito: Máximo local, máximo global
#             #Se a medida de qualidade for igual à anterior (por duas vezes?)
#             #FALSE
#     else:
#         continua = False
#         break

    if orcamento > 0 : #Se ainda tem orçamento pra gastar
        if cv_atual < 10:
            if (f1_atual >= f1_anterior) and (igual < 3): #Selecionam-se novos pares para rotulação
    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                if f1_atual == f1_anterior:
                    igual += 1
                    print("igual: {0}".format(igual))
                elif (f1_atual > f1_anterior) and (igual > 0):
                    igual = 0

                continua = True
    #             break
            else: #TODO ERRADO! AJEITAR!
                max_local += 1
                #Se entrar aqui, tem que remover os últimos carinhas
                #que entraram no conjunto de treinamento

                #Acho que esses devem ser extraídos para o conjunto PM, não apenas descartados
                conj_treino.drop(conj_treino.tail(2).index,inplace=True)

    #             print("Antes do max_local")

    #             print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))
    #             print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

                print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                if(max_local <= 1):
                    f1_atual = f1_anterior
    #                 print("Depois do max_local")
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                else:
                    f1_atual = f1_anterior
    #                 print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
                    continua = False
                    break
                #Pensar a respeito: Máximo local, máximo global
                #Se a medida de qualidade for igual à anterior (por duas vezes?)
    else:
        continua = False
        break
    
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
    print("igual: {0}".format(igual))

Início
orcamento: 80
Quantidade de itens no conjunto de treinamento: 20
F1 anterior: 0.8099999999999999 - F1 atual: 0.9266666666666666
igual: 0
F1 anterior: 0.9266666666666666 - F1 atual: 0.8933333333333333
igual: 0
F1 anterior: 0.8933333333333333 - F1 atual: 1.0
igual: 0
F1 anterior: 1.0 - F1 atual: 0.9857142857142858
F1 anterior: 1.0 - F1 atual: 1.0
igual: 0
F1 anterior: 1.0 - F1 atual: 0.9857142857142858


In [53]:
print("F1 anterior: {0} - F1 atual: {1}".format(f1_anterior, f1_atual))
print("Quantidade de itens no conjunto de treinamento: {}".format(len(conj_treino)))

print("F1: {0}".format(f1_atual))

print("Desvio padrão: {0}".format(std_atual))

print("Coeficiente de variação: {0}%.".format(str(cv_atual)[:5]))

duplicatas = [i for i in conj_treino.duplicata if i == True]
percentage = float(duplicatas.count(True)/float(len(conj_treino))*100)
print("{0}% de duplicatas.".format(str(percentage)[:6]))

naoduplicatas = [i for i in conj_treino.duplicata if i == False]
percentage2 = float(naoduplicatas.count(False)/float(len(conj_treino))*100)
print("{0}% de nao-duplicatas.".format(str(percentage2)[:6]))
print("orcamento: {0} - jan_inic: {1} - jan_fin: {2}".format(orcamento, jan_inic, jan_fin))

F1 anterior: 1.0 - F1 atual: 1.0
Quantidade de itens no conjunto de treinamento: 26
F1: 1.0
Desvio padrão: 0.02857142857142858
Coeficiente de variação: 2.941%.
46.153% de duplicatas.
53.846% de nao-duplicatas.
orcamento: 70 - jan_inic: 42 - jan_fin: 45


# Atenção!
## Depois ver como os índices de conj_treino interagem com o conjunto original

In [100]:
conj_treino

,duplicata,title,artist,track01,track02,track03,track10,track11
"(105349, 105518)",0.0,1.000000,0.0,0.500000,0.454545,0.818182,0.454545,0.818182
"(104289, 105349)",0.0,0.666667,0.0,0.400000,1.000000,0.454545,1.000000,0.454545
"(105349, 3710)",0.0,1.000000,0.0,0.416667,0.909091,0.500000,0.909091,0.500000
"(101480, 107779)",0.0,1.000000,0.0,0.375000,0.409091,0.400000,0.409091,0.400000
"(101480, 104396)",0.0,1.000000,0.0,0.461538,0.409091,0.833333,0.409091,0.833333
"(101480, 104396)",0.0,1.000000,0.0,0.461538,0.409091,0.833333,0.409091,0.833333


In [118]:
a = conj_treino.sample(n = 1, random_state=seed)
a
type(a.index.values)

numpy.ndarray

In [117]:
vetor_sel = pc_vetores.loc[a.index.values]
vetor_sel
# type(vetor_sel)

,,title,artist,track01,track02,track03,track10,track11
elemento1,elemento2,,,,,,,
105349,105518,1.0,0.0,0.5,0.454545,0.818182,0.454545,0.818182


In [61]:
conj_treino = conj_treino.append(vetor_sel)
conj_treino

,artist,duplicata,title,track01,track02,track03,track10,track11
"(105349, 105518)",0.0,0.0,1.000000,0.500000,0.454545,0.818182,0.454545,0.818182
"(104289, 105349)",0.0,0.0,0.666667,0.400000,1.000000,0.454545,1.000000,0.454545
"(105349, 3710)",0.0,0.0,1.000000,0.416667,0.909091,0.500000,0.909091,0.500000
"(101480, 107779)",0.0,0.0,1.000000,0.375000,0.409091,0.400000,0.409091,0.400000
"(101480, 104396)",0.0,0.0,1.000000,0.461538,0.409091,0.833333,0.409091,0.833333
"(101480, 104396)",0.0,0.0,1.000000,0.461538,0.409091,0.833333,0.409091,0.833333
"(105349, 105518)",0.0,NaN,1.000000,0.500000,0.454545,0.818182,0.454545,0.818182
"(105349, 105518)",0.0,NaN,1.000000,0.500000,0.454545,0.818182,0.454545,0.818182


In [149]:
jan_inic = 0 
jan_fin = k

cont = 0

entrouExcept = False

while cont < 20:

#     print("orçamento atual: {0}.".format(orcamento))
    
    print("jan_inic: {0} - jan_fin: {1}.".format(jan_inic,jan_fin))          
    
    #Desliza-se a janela
    grupo = ndup.iloc[jan_inic:jan_fin] # three rows of dataframe
#     a = grupo.sample(n = 1, random_state=500)
#     print(a)

#     #SELEÇÃO DA POSSÍVEL NÃO-DUPLICATA
#     #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
#     entrouExcept = False
    
#     try:
    
#         id_row_sel = grupo['min'].idxmax() #Retorna o índice da linha com maior valor da coluna ('min') especificada
    
#     except ValueError:
        
#         entrouExcept = True
        
#         #Seleciona um aleatoriamente
#         a = grupo.sample(n = 1, random_state=500) 
#         id_row_sel = a.index.values #Retorna o índice da linha com maior valor da coluna ('min') especificada
#         id_row_sel = tuple(map(tuple, id_row_sel))
#         print(type(id_row_sel))
#         id_row_sel = id_row_sel[0]
#         print(id_row_sel)
#     except:
#         print('Eita!')

#     #Rotula o par, retira do conjunto U (Nesse caso "pc_vetores" terá os pares retirados ao final da composição do conjunto de treinamento)
#     duplicata = grupo.loc[id_row_sel, 'duplicata'] #Status do par
    
#     print("tipoDoId: {3} - entrouExcept: {2} - id: {0} - duplicata: {1}.".format(id_row_sel, duplicata, entrouExcept, type(id_row_sel)))          

#     vetor_sel = pc_vetores.loc[id_row_sel]
#     conj_treino = conj_treino.append(vetor_sel) #Adicionando o vetor selecionado ao conjunto treino
#     conj_treino.loc[id_row_sel, 'duplicata'] = duplicata #Adicionando o status de duplicata do vetor selecionado ao conjunto treino
    
#     #SELEÇÃO DA POSSÍVEL DUPLICATA
#     #Seleciona-se o par com menor quantidade de pares em concordância (talvez selecionar o que teve maior concordância também?) para rotulação
    
#     entrouExcept = False
      
    #Atualização da janela
    jan_inic = jan_inic + k
    jan_fin = jan_fin + k
    
    orcamento -= 1 #Têm que ser atualizado no povoamento inicial também
    
    cont += 1
    print(grupo)
 

jan_inic: 0 - jan_fin: 3.
                     duplicata  qtdAlg       min       max       med
elemento1 elemento2                                                 
104289    105349           0.0       1  0.488872  0.488872  0.488872
102831    105349           0.0       1  0.492994  0.492994  0.492994
105349    105518           0.0       1  0.496958  0.496958  0.496958
jan_inic: 4 - jan_fin: 7.
                     duplicata  qtdAlg       min       max       med
elemento1 elemento2                                                 
101480    107779           0.0       1  0.506742  0.506742  0.506742
105349    3710             0.0       1  0.507576  0.507576  0.507576
          2004             0.0       1  0.507576  0.507576  0.507576
jan_inic: 8 - jan_fin: 11.
                     duplicata  qtdAlg       min       max       med
elemento1 elemento2                                                 
101480    104396           0.0       1  0.515396  0.515396  0.515396
102477    104117        

In [147]:
len(ndup)

758